# Verifinger

We used the Verifinger SDK (free downloadable version) available at - https://www.neurotechnology.com/download.html#demo, so download it.

### Step 1: Generate Score Matrix

1. To enable permissions run- `chmod +x FingersAlgorithmDemo.app`

2. Then we run the FingersAlgorithmDemo.app executable

    For Linux:
   `./FingersAlgorithmDemo` or 

    For Mac:
    `open FingersAlgorithmDemo.app`

3. First we enrolled our Session1 data.

4. Then we ran a script to obtain score_matrix of matching Session1 with Session2 by performing identification with each possible combination

### Step 2: Calculate Match Scores & Plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
def load_scores(data_path):
    scores_matrix = []
    with open(f"{data_path}/scores_matrix.txt", "r") as f:
        for line in f:
            row = list(map(int, line.strip().split(",")))
            scores_matrix.append(row)
    scores_matrix = np.array(scores_matrix)

    genuine_scores = np.diag(scores_matrix)
    genuine_scores = np.array(genuine_scores)

    imposter_scores = []
    used_pairs = set()

    while len(imposter_scores) < 1000:
        i, j = np.random.randint(0, 100, 2)
        if i != j and (i, j) not in used_pairs:
            imposter_scores.append(scores_matrix[i][j])
            used_pairs.add((i, j))
                
    imposter_scores = np.array(imposter_scores) 

    return genuine_scores, imposter_scores

In [ ]:
def compute_arrays(data_path):
    genuine_scores, imposter_scores = load_scores(data_path)

    min_score = min(genuine_scores.min(), imposter_scores.min())
    max_score = max(genuine_scores.max(), imposter_scores.max())
    thresholds = np.arange(min_score, max_score, 5)

    # Initialize metrics
    FAR = []  # False Acceptance Rate
    FRR = []  # False Rejection Rate
    TPR = []  # True Positive Rate    
    FPR = []  # False Positive Rate

    for threshold in thresholds:
        false_rejects = sum(score < threshold for score in genuine_scores)
        true_accepts = sum(score >= threshold for score in genuine_scores)
        
        false_accepts = sum(score >= threshold for score in imposter_scores)
        true_rejects = sum(score < threshold for score in imposter_scores)
        
        FAR.append(false_accepts / len(imposter_scores))
        FRR.append(false_rejects / len(genuine_scores))
        TPR.append(true_accepts / len(genuine_scores))
        FPR.append(false_accepts / len(imposter_scores))

    return np.array(thresholds), np.array(FAR), np.array(FRR), np.array(TPR), np.array(FPR)


In [ ]:
def plot_roc_curves(ocr_paths, labels=[None, None, None], colors=['blue', 'green', 'orange']):
    plt.figure(figsize=(6, 6))

    for ocr_path, label, color in zip(ocr_paths, labels, colors):
        thresholds, FAR, FRR, TPR, FPR = compute_arrays(ocr_path)
        roc_auc = auc(FPR, TPR)
        
        plt.plot(FPR, TPR, color=color, lw=2,
                 label=f'{label} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', lw=1.2)

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Acceptance Rate (FAR)', fontsize=14)
    plt.ylabel('True Positive Rate (TPR)', fontsize=14)

    plt.legend(loc='lower right', fontsize=13, frameon=True)
    plt.grid(linestyle='--', alpha=0.4)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_far_frr(ocr_paths, titles, colors=['blue', 'green', 'orange']):
    plt.figure(figsize=(6, 6))

    for i in range(len(ocr_paths)):
        thresholds, FAR, FRR, TPR, FPR = compute_arrays(ocr_paths[i])
        eer_index = np.argmin(np.abs(FAR - FRR))
        eer_threshold = thresholds[eer_index]
        EER = (FAR[eer_index] + FRR[eer_index]) / 2

        plt.plot(FRR, FAR, label=f"{titles[i]} (EER={EER:.2f})", color=colors[i])

        plt.scatter(FRR[eer_index], FAR[eer_index],
                    s=40, color=colors[i], marker="o", edgecolor='k', zorder=5)

    plt.plot([0, 1], [0, 1], 'k--', alpha=0.4)

    plt.scatter([], [], s=40, color="white", edgecolor="k", label="EER point")

    plt.xlabel("False Rejection Rate (FRR)", fontsize=14)
    plt.ylabel("False Acceptance Rate (FAR)", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.legend(fontsize=13, loc="upper right", frameon=True)
    
    plt.tight_layout()
    plt.show()



# DeepPrint

We used the DeepPrint Library code available at - https://github.com/tim-rohwedder/fixed-length-fingerprint-extractors, so clone the repository to run the below code.

### Step 1: Generate Embeddings

We slightly changed their library code to allow recognition of our dataset of .tif images

In [ ]:
import matplotlib.pyplot as plt
from flx.data.image_loader import ImageLoader
import torch
import torchvision.transforms.functional as VTF
import PIL
from IPython.display import display
import os
from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex, get_DeepPrint_TexMinu, DeepPrintExtractor
from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size
from flx.data.embedding_loader import EmbeddingLoader
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.scripts.generate_benchmarks import create_verification_benchmark
from sklearn.metrics import auc
import numpy as np

In [ ]:
# To load the pre-trained model parameters use num_training_subjects=8000
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=8000, num_dims=256)

# To load the pre-trained model parameters use
MODEL_DIR: str = os.path.abspath("<MODEL>") # Path to the directory containing the model parameters
extractor.load_best_model(MODEL_DIR)

DATASET_PATH: str = os.path.abspath("<DATASET>")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_PATH),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )


image_dataset: Dataset = Dataset(image_loader, image_loader.ids)

texture_embeddings, minutia_embeddings = extractor.extract(image_dataset)

In [ ]:
texture_embeddings.save("embeddings/<DATAPATH>_texture_emb")
minutia_embeddings.save("embeddings/<DATAPATH>_minutia_emb")

### Step 2: Calculate Match Scores & Plot

Now we load the embeddings and calculate match scores and plot ROC and FARvsFRR Curves.

In [ ]:
### LOAD DATASET

def load_dataset(path: str) -> Dataset:
    """Load the dataset from the given path."""
    DATASET_PATH: str = os.path.abspath("<DATASET>"+path)
    # We will use the SFingeLoader to load the images from the dataset
    image_loader = TransformedImageLoader(
            images=SFingeLoader(DATASET_PATH),
            poses=None,
            transforms=[
                LazilyAllocatedBinarizer(5.0),
                pad_and_resize_to_deepprint_input_size,
            ],
        )

    image_dataset: Dataset = Dataset(image_loader, image_loader.ids)
    return image_dataset

In [ ]:
### LOAD EMBEDDINGS

def load_embeddings(path: str) -> EmbeddingLoader:
    texture_embeddings = EmbeddingLoader.load(f"embeddings/{path}_texture_emb")
    minutia_embeddings = EmbeddingLoader.load(f"embeddings/{path}_minutia_emb")
    return texture_embeddings, minutia_embeddings

In [ ]:
def get_scores(path: str):
    image_dataset = load_dataset(path)
    texture_embeddings, minutia_embeddings = load_embeddings(path)
    embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)
    matcher = CosineSimilarityMatcher(EmbeddingLoader.combine(texture_embeddings, minutia_embeddings))
    # Can change mode of verifier
    # matcher = CosineSimilarityMatcher(texture_embeddings)
    # matcher = CosineSimilarityMatcher(minutia_embeddings)

    NUM_IMPRESSIONS_PER_SUBJECT = 2 # Can modify according to how many images have been captured
    benchmark = create_verification_benchmark(
    subjects=list(range(image_dataset.num_subjects)),
    impressions_per_subject=list(range(NUM_IMPRESSIONS_PER_SUBJECT)))

    results = benchmark.run(matcher)

    genuine_scores = results.get_mated_scores()
    imposter_scores = results.get_non_mated_scores()

    return genuine_scores, imposter_scores, results

In [ ]:
def compute_arrays(genuine_scores, imposter_scores):
    min_score = min(genuine_scores.min(), imposter_scores.min())
    max_score = max(genuine_scores.max(), imposter_scores.max())
    thresholds = np.linspace(min_score, max_score, num=100)

    FAR = []  # False Acceptance Rate
    FRR = []  # False Rejection Rate
    TPR = []  # True Positive Rate    
    FPR = []  # False Positive Rate

    for threshold in thresholds:
        false_rejects = sum(score < threshold for score in genuine_scores)
        true_accepts = sum(score >= threshold for score in genuine_scores)
        
        false_accepts = sum(score >= threshold for score in imposter_scores)
        true_rejects = sum(score < threshold for score in imposter_scores)
        
        FAR.append(false_accepts / len(imposter_scores))
        FRR.append(false_rejects / len(genuine_scores))
        TPR.append(true_accepts / len(genuine_scores))
        FPR.append(false_accepts / len(imposter_scores))

    return np.array(thresholds), np.array(FAR), np.array(FRR), np.array(TPR), np.array(FPR)

In [ ]:
def plot_roc_curves(ocr_paths, labels=None, colors=None):
    plt.figure(figsize=(6, 6))

    for ocr_path, label, color in zip(ocr_paths, labels, colors):
        genuine_scores, imposter_scores, results = get_scores(ocr_path)
        thresholds, FAR, FRR, TPR, FPR = compute_arrays(genuine_scores, imposter_scores)
        roc_auc = auc(FPR, TPR)
        plt.plot(FPR, TPR, color=color, lw=2,
                 label=f'{label} (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', lw=1.2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Acceptance Rate (FAR)', fontsize=14)
    plt.ylabel('True Positive Rate (TPR)', fontsize=14)
    plt.legend(loc='lower right', fontsize=13, frameon=True)
    plt.grid(linestyle='--', alpha=0.4)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_far_frr(ocr_paths, titles, colors=None):
    plt.figure(figsize=(6, 6))
    for i in range(len(ocr_paths)):
        genuine_scores, imposter_scores, results = get_scores(ocr_paths[i])
        thresholds, FAR, FRR, TPR, FPR = compute_arrays(genuine_scores, imposter_scores)

        eer_index = np.argmin(np.abs(FAR - FRR))
        eer_threshold = thresholds[eer_index]
        EER = (FAR[eer_index] + FRR[eer_index]) / 2

        plt.plot(FRR, FAR, label=f"{titles[i]} (EER={EER:.2f})", color=colors[i])

        plt.scatter(FRR[eer_index], FAR[eer_index],
                    s=40, color=colors[i], marker="o", edgecolor='k', zorder=5)

    plt.plot([0, 1], [0, 1], 'k--', alpha=0.4)

    plt.scatter([], [], s=40, color="white", edgecolor="k", label="EER point")

    plt.xlabel("False Rejection Rate (FRR)", fontsize=14)
    plt.ylabel("False Acceptance Rate (FAR)", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.legend(fontsize=13, loc="upper right", frameon=True)
    
    plt.tight_layout()
    plt.show()